# Run a job to get optimisation results

## Configure Watson Machine Learning client

In [1]:
# Config from previous deployment
deployment_uid = '4b889fed-2354-40ee-8d6a-629264858759'
space_uid = 'ba8c7337-02aa-4048-96fd-ffdd2ab82e76'

In [2]:
## Create a WML v4 client
from ibm_watson_machine_learning import APIClient

wml_credentials = {
      "apikey": "<api_key>",
      "url": "https://us-south.ml.cloud.ibm.com"
}

client = APIClient(wml_credentials)
print(client.version)

1.0.45


In [3]:
# Set client's default space
client.set.default_space(space_uid)

'SUCCESS'

## Prepare input data

In [4]:
import pandas as pd 
  
# initialize list of lists 
input_params = pd.DataFrame([{"axe_strength": 1, "lc_strength": 1, "ma_strength": 1, "build_time_limit_seconds": 2_592_000}])
print(input_params)

   axe_strength  lc_strength  ma_strength  build_time_limit_seconds
0             1            1            1                   2592000


## Create optimisation job

In [5]:
# Prepare inputs and outputs expectations for solver

# Using inline tabular data
# https://dataplatform.cloud.ibm.com/docs/content/DO/WML_Deployment/ModelIODataDefn.html?context=wdp&audience=wdp
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [{
      "id":"params.csv", # This is required by my model
      "values" : input_params
    }],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [{
        "id":".*\.csv" # My model is configured to output CSVs for results
    }]
}

job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print(job_uid)

48eae6aa-f27a-4d94-a71a-05db545f7717


## Wait for job to complete

In [6]:
# Job can take a while to run - a machine with CPLEX needs to start first
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
completed


## Print results

In [7]:
from pprint import pprint

pprint(job_details['entity']['decision_optimization']['output_data'])

[{'fields': ['Name', 'Value'],
  'id': 'kpis.csv',
  'values': [['Total negative attack strength', -682455],
             ['Barracks build time', 1799760],
             ['Hall of Order build time', 133200]]},
 {'fields': ['axe',
             'food',
             'lc',
             'ma',
             'ram',
             'serk',
             'time_in_days',
             'time_in_seconds',
             'total_attack_strength'],
  'id': 'solution.csv',
  'values': [[4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400, 683050],
             [4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400, 683050],
             [4985, 20596, 1725, 1491, 250, 1, 21.55, 1861800, 673025],
             [4978, 20580, 1724, 1490, 250, 1, 21.53, 1860360, 672430],
             [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960, 682455],
             [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960, 682455]]}]


In [9]:
output = job_details['entity']['decision_optimization']['output_data']

In [34]:
# Re-formatting for future parsing, next time output a JSON instead of a CSV
results = []
for o in output:    
    if o['id'] == "solution.csv":
        fields = o['fields']
        solutions = o['values']
        for s in solutions:
            r = {}
            for i, v in enumerate(s):
                r[fields[i]] = s[i]
            results.append(r)
print(results)

[{'axe': 4814, 'food': 20596, 'lc': 1664, 'ma': 1442, 'ram': 250, 'serk': 111, 'time_in_days': 22.39, 'time_in_seconds': 1934400, 'total_attack_strength': 683050}, {'axe': 4814, 'food': 20596, 'lc': 1664, 'ma': 1442, 'ram': 250, 'serk': 111, 'time_in_days': 22.39, 'time_in_seconds': 1934400, 'total_attack_strength': 683050}, {'axe': 4985, 'food': 20596, 'lc': 1725, 'ma': 1491, 'ram': 250, 'serk': 1, 'time_in_days': 21.55, 'time_in_seconds': 1861800, 'total_attack_strength': 673025}, {'axe': 4978, 'food': 20580, 'lc': 1724, 'ma': 1490, 'ram': 250, 'serk': 1, 'time_in_days': 21.53, 'time_in_seconds': 1860360, 'total_attack_strength': 672430}, {'axe': 4807, 'food': 20580, 'lc': 1663, 'ma': 1441, 'ram': 250, 'serk': 111, 'time_in_days': 22.37, 'time_in_seconds': 1932960, 'total_attack_strength': 682455}, {'axe': 4807, 'food': 20580, 'lc': 1663, 'ma': 1441, 'ram': 250, 'serk': 111, 'time_in_days': 22.37, 'time_in_seconds': 1932960, 'total_attack_strength': 682455}]


### Full output

In [8]:
job_details

{'entity': {'decision_optimization': {'input_data': [{'fields': ['axe_strength',
      'lc_strength',
      'ma_strength',
      'build_time_limit_seconds'],
     'id': 'params.csv',
     'values': [[1, 1, 1, 2592000]]}],
   'output_data': [{'fields': ['Name', 'Value'],
     'id': 'kpis.csv',
     'values': [['Total negative attack strength', -682455],
      ['Barracks build time', 1799760],
      ['Hall of Order build time', 133200]]},
    {'fields': ['axe',
      'food',
      'lc',
      'ma',
      'ram',
      'serk',
      'time_in_days',
      'time_in_seconds',
      'total_attack_strength'],
     'id': 'solution.csv',
     'values': [[4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400, 683050],
      [4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400, 683050],
      [4985, 20596, 1725, 1491, 250, 1, 21.55, 1861800, 673025],
      [4978, 20580, 1724, 1490, 250, 1, 21.53, 1860360, 672430],
      [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960, 682455],
      [4807, 20580, 1663